# 1. pandas与模型代码的接口
- 模型开发的通常工作流是使用Pandas进行数据加载和清洗，然后切换到建模库进行建模。
- pandas与其它分析库通常是靠numpy的数组联系起来的
    - df.values：将DataFrame转换为numpy数组
    - pd.DataFrame(df.values, columns=)
    - 最好当数据类型一致时使用values属性；若数据不均匀，返回python对象的ndarray
    - df.loc['col'].values：选取列的子集
- 替换category列为虚变量
    - data.drop('category', axis=1).join(pd.get_dummies(data.category, prefix='category'))：先删除category列，再添加虚变量列

# 2. 用Patsy创建模型描述
- Pasty，使用简短的字符串“公式语法”描述统计模型，适合描述statsmodels的线性模型
- y, x = patsy.dmatrices('y ~ x1 + x2', data)
    - 返回DesignMatrix实例
    - y对应~左边因变量y，x对应~右边的所有自变量
    - 'y ~ f(x) + 0'则返回的x中不显示intercept列
    - x,y可直接传递到算法中，
    - 如执行最小二乘回归：
      >>coef, resid, _, _ = np.linalg.lstsq(x, y)
      >>coef=pd.Series(coef.squeeze(),index=x.design_info.column_names)
      返回各自变量的系数和常数项
    - 'y ~ x1 + x2'中的'+'并不是加法的意义，只表示x1和x2的集合；若想赋予其加法意义，必须用特定函数将它们封装起来，'y ~ I(x0 + x1)'
- 用patsy公式进行数据转换
    - 'y ~ x0 + np.log(np.abs(x1) + 1)'
    - Patsy内置函数：'y ~ standardize(x0) + center(x1)'
    - 用另一个数据集评估检测拟合的模型，也称为状态转换：
      >>new_x = patsy.build_design_matrices([x.design_info], new_data)
    -
- patsy转换分类数据
    - patsy公式中的非数值数据会默认转换为虚变量（进行分类）
    - 使用C函数，数值列可截取为分类量
        - 'v2 ~ C(key2)'，返回结果中C(key2)[T.元素1]...
    - 'v2 ~ k1:k2'，返回k1与k2相交部分（如真真得真，真假为假）
 
# 3. statsmodels介绍
- python进行拟合多种统计模型、进行统计试验和数据探索可视化的库，包含很多经典的统计方法（线性模型、线性混合效应模型、方差方法分析、时间序列过程和状态空间模型、广义矩估计），但没有贝叶斯方法和机器学习模型。
- 估计线性模型
    - 有两种不同的接口：基于数组(statsmodels.api)和基于公式(statsmodels.formula.api)
    - 基于数组sm （X、y为数组）
        - 先添加一个截距列：X_model = sm.add_constant(X)
        - 选定模型：model = sm.OLS(y, X_model)  普通最小二乘回归
        - 拟合：results = model.fit()
        - 拟合结果：results.params——拟合参数，results.summary()——所有拟合结果与说明
        - 根据模型预测：results.predict(X_model)
    - 基于公式smf （X、y存放于一个DataFrame）
        - 无需添加截距项
        - 选模型与拟合：results = smf.ols('patsy公式', data).fit()
        - 拟合结果：results.params——拟合参数、results.tvalues——t检验、results.pvalues——p检验
        - 根据模型预测：results.predict(data)
- 估计时间序列过程
    - 包括自回归过程、卡尔曼滤波和其他态空间模型、多元自回归模型

# 4. scikit-learn介绍
- python机器学习库
- statsmodels和scikit-learn通常不能接受缺失数据，需缺失数据补全fillna
- from sklearn.linear_model import LogisticRegression
  >>选模型：model = LigisticRegression()  
  >>学习：model.fit(x_train, y_train)
  >>预测：y_predict = model.predict(x_test)
  >>计算预测准确率：(y_true == y_predict).mean()
- 避免对训练数据过拟合的方法，如交叉验证

In [8]:
import pandas as pd
import numpy as np

data = pd.DataFrame({'x0': [1, 2, 3, 4, 5],
                     'x1': [0.01, 0.2, -0.1, -2, 1],
                     'y': [-1.1, 2, 0, 2.2, -3]})
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'])

dummies = pd.get_dummies(data.category, prefix='category')
data_with_dummies = data.drop('category', axis=1).join(dummies)
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.1,1,0
1,2,0.20,2.0,0,1
2,3,-0.10,0.0,1,0
3,4,-2.00,2.2,1,0
4,5,1.00,-3.0,0,1


In [44]:
# patsy

import pandas as pd
import numpy as np
import patsy

data = pd.DataFrame({'x0': [1, 2, 3, 4, 5],
                     'x1': [0.01, 0.2, -0.1, -2, 1],
                     'y': [-1.1, 2, 0, 2.2, -3]})
y, X = patsy.dmatrices('y ~ x0 + x1', data)
y
X
np.asarray(y)
np.asarray(X)

patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]
#最小二乘法
coef, resid, _, _ = np.linalg.lstsq(X, y, rcond=-1)
coef
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)
coef


data2 = pd.DataFrame({'x': [1, 2, 3, 4, 5],
                      'y': [-1, -2.1, -3, -4.01, -4.9]})
y, x = patsy.dmatrices('y ~ x', data2)
coef2, resid2, _, _ = np.linalg.lstsq(x, y, rcond=-1)
coef2
coef2 = pd.Series(coef2.squeeze(), index=x.design_info.column_names)
coef2   #返回各自变量的系数


#用patsy公式进行数据转换
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)
X

y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)
X

#数据转换（状态转换）
new_data = pd.DataFrame({'x0': [6, 7, 8, 9],
                         'x1': [2.2, -1.1, 2, 0.3],
                         'y': [1, 3, 4, 2]})
new_X = patsy.build_design_matrices([X.design_info], new_data)
new_X

#公式中的加号意义
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)   #加法
X


#
data3 = pd.DataFrame({'key1': list('aabbabab'),
                      'key2': [0, 1, 0, 1, 0, 1, 0, 0],
                      'v1': [1, 2, 3, 4, 5, 6, 7, 8],
                      'v2': [-1, 0, 2.5, -0.5, 4, -1.2, 0.2, -1.7]})
#非数值数据默认转换为虚变量
y, X = patsy.dmatrices('v2 ~ key1 + 0', data3)
X

#C函数，数值列可截取为分类量
y, X = patsy.dmatrices('v2 ~ C(key2)', data3)
X

data3['key2'] = data3['key2'].map({0: 'zero', 1: 'one'})
data3
y, X = patsy.dmatrices('v2 ~ key1 + key2', data3)
X

#key1:key2相交部分
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data3)
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

In [ ]:
# statsmodels
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf


#线性模型
#首先生成一些正态分布的随机数据
def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size,
    return mean + np.sqrt(variance) * np.random.randn(*size)

np.random.seed(12345)
N = 100
X = np.c_[dnorm(0, 0.4, size=N),
          dnorm(0, 0.6, size=N),
          dnorm(0, 0.2, size=N)]
eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]
y = np.dot(X, beta) + eps

#再数组拟合
X_model = sm.add_constant(X)  #添加截距列
model = sm.OLS(y, X_model)  #sm.OLS类可拟合一个普通最小二乘回归
results = model.fit()
results.params  #拟合参数
results.summary()  #所有拟合结果
results.predict(X_model[:5])

#公式拟合
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])
data['y'] = y
results2 = smf.ols('y ~ col0 + col1 + col2', data).fit()
results2.params
results2.tvalues  #t检验
results2.predict(data[:5])  #根据模型计算预测值


#估计时间序列过程
#用自回归结构和噪声来模拟时间序列数据
init_x = 4
values = [init_x, init_x]
N =1000

b0 = 0.8
b1 = -0.4
noise = dnorm(0, 0.1, N)
for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)

MAXLAGS = 5
model = sm.tsa.AR(values)
results = model.fit(MAXLAGS)
results.params